In [1]:
import datetime 
#from gensim.models import Word2Vec 
from itertools import combinations 
import random 
import pandas as pd 
from sklearn.calibration import label_binarize 
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier, VotingClassifier 
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC 
from sklearn.naive_bayes import MultinomialNB 
from sklearn.ensemble import GradientBoostingClassifier 
from sklearn.preprocessing import StandardScaler, MinMaxScaler 
from sklearn.metrics import accuracy_score, balanced_accuracy_score, classification_report, f1_score, roc_auc_score 
from sklearn.decomposition import LatentDirichletAllocation 
from sklearn.metrics.pairwise import cosine_similarity 
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score
from sklearn.preprocessing import LabelBinarizer
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
import numpy as np 
from transformers import BertTokenizer, BertModel 
#import shap 
import matplotlib.pyplot as plt 
from imblearn.over_sampling import SMOTE 
import seaborn as sns 
from ast import literal_eval
#from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, cross_val_predict
from langdetect import detect
import pickle
import spacy
import re
import string
from gensim.models import Word2Vec
from spacy.lang.ru.stop_words import STOP_WORDS as stop_words
from sklearn.inspection import permutation_importance
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.decomposition import PCA



/Users/pratappokkharel/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
comments_df = pd.read_csv('comments_f2v_fts.csv')
videos_df = pd.read_csv('vid_fin.csv')

Отбиарем 1000 размеченных комментариев

In [3]:
comments_df = comments_df[:1000]

In [4]:
# Функция для подсчета иностранных слов
def count_foreign_words(text):
    if isinstance(text, str):
        foreign_word_count = 0
        words = text.split()
        for word in words:
            try:
                lang = detect(word)
                if lang != 'ru':
                    foreign_word_count += 1
            except LangDetectException:
                continue  # Пропускаем слово, если не удалось определить язык
        return foreign_word_count
    return 0

In [5]:
comments_df['Foreign_Words'] = comments_df['Processed_Text'].apply(count_foreign_words)

In [6]:
# Загрузка модели spaCy для русского языка
nlp = spacy.load("ru_core_news_sm")
comments_df['Text'] = comments_df['Text'].fillna('').astype(str)
# Функция для предобработки текста
def preprocess_text(text):
    # Обработка NaN значений
    if pd.isnull(text):
        return ""
    # Приведение к нижнему регистру
    text = text.lower()
    # Удаление ссылок и других ненужных символов
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\@w+|\#', '', text)
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Токенизация и лемматизация
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if token.lemma_ not in stop_words and token.is_alpha]
    return ' '.join(tokens)

# Применение предобработки к каждому тексту
comments_df['Processed_Text'] = comments_df['Text'].apply(preprocess_text)

# Создание корпуса из текстов комментариев
corpus = comments_df['Processed_Text'].tolist()

# Разделение текста на предложения
sentences = [text.split() for text in corpus]

# Создание модели Word2Vec
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

def get_comment_vector(comment, model):
    words = comment.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

    
def extract_additional_features(comment):
    length = len(comment)
    uppercase_count = sum(1 for c in comment if c.isupper())
    punctuation_count = sum(1 for c in comment if c in string.punctuation)
    emoji_count = len(re.findall(r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0]+', comment))
    return np.array([length, uppercase_count, punctuation_count, emoji_count])

# Преобразование всех комментариев в векторы и добавление дополнительных признаков
def get_combined_vector(comment, model):
    word_vector = get_comment_vector(comment, model)
    additional_features = extract_additional_features(comment)
    combined_vector = np.concatenate((word_vector, additional_features), axis=None)
    return combined_vector

# Преобразование всех комментариев в векторы
comments_df['Vector'] = comments_df['Processed_Text'].apply(lambda x: get_comment_vector(x, word2vec_model))

In [7]:
comments_df['Vector_features'] = comments_df['Text'].apply(lambda x: get_combined_vector(preprocess_text(x), word2vec_model))

In [8]:
comm_features_label = pd.read_csv('vid_23.csv')

In [9]:
comments_df_fin = comments_df.iloc[:1000]

In [10]:
comments_df_fin['Label'] = comm_features_label['Label']

In [30]:
new_features = ['LDA_Similarity', 'Thread_Similarity', 'Videosub_Similarity', 'Sentiment_Label', 'Sentiment_Score']

In [31]:
comments_df_fin[new_features] = comm_features_label[new_features]

Теперь построим модель классификации

# Word2vec

In [12]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

# Разделение данных на признаки и целевую переменную
X = np.vstack(comments_df_fin['Vector'].values)
y = comments_df_fin['Label']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создание и обучение моделей
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
lr_model = LogisticRegression(max_iter=1000, random_state=42, multi_class='multinomial', solver='lbfgs')
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
svc_model = SVC(probability=True, random_state=42)
nb_model = MultinomialNB()

models = {
    "Random Forest": rf_model,
    "Logistic Regression": lr_model,
    "Gradient Boosting": gb_model,
    "SVC": svc_model,
#    "Naive Bayes": nb_model
}

# Обучение моделей
for name, model in models.items():
    model.fit(X_train, y_train)

# Предсказание на тестовых данных и оценка моделей
def evaluate_model(y_test, y_pred, y_proba, model_name):
    accuracy = accuracy_score(y_test, y_pred)
    f1_micro = f1_score(y_test, y_pred, average='micro')
    f1_macro = f1_score(y_test, y_pred, average='macro')
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test_binarized = lb.transform(y_test)
    roc_auc_ovo = roc_auc_score(y_test_binarized, y_proba, multi_class='ovo')
    roc_auc_ovr = roc_auc_score(y_test_binarized, y_proba, multi_class='ovr')
    print(f"{model_name}:")
    print("Accuracy:", accuracy)
    print("F1 Score (Micro):", f1_micro)
    print("F1 Score (Macro):", f1_macro)
    print("ROC AUC (OvO):", roc_auc_ovo)
    print("ROC AUC (OvR):", roc_auc_ovr)
    print(classification_report(y_test, y_pred))
    print()

# Вывод результатов для каждой модели
for name, model in models.items():
    y_pred = model.predict(X_test)
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test)
    else:
        # For models without predict_proba (e.g., SVC without probability=True)
        ovo_classifier = OneVsOneClassifier(model)
        ovo_classifier.fit(X_train, y_train)
        y_proba = ovo_classifier.decision_function(X_test)
    evaluate_model(y_test, y_pred, y_proba, name)
    
# Предсказание на тестовых данных и оценка моделей
def evaluate_model(y_test, y_pred, y_proba, model_name):
    accuracy = accuracy_score(y_test, y_pred)
    f1_micro = f1_score(y_test, y_pred, average='micro')
    f1_macro = f1_score(y_test, y_pred, average='macro')
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test_binarized = lb.transform(y_test)
    roc_auc_ovo = roc_auc_score(y_test_binarized, y_proba, multi_class='ovo')
    roc_auc_ovr = roc_auc_score(y_test_binarized, y_proba, multi_class='ovr')
    print(f"{model_name}:")
    print("Accuracy:", accuracy)
    print("F1 Score (Micro):", f1_micro)
    print("F1 Score (Macro):", f1_macro)
    print("ROC AUC (OvO):", roc_auc_ovo)
    print("ROC AUC (OvR):", roc_auc_ovr)
    print(classification_report(y_test, y_pred))
    print()

# Вывод результатов для каждой модели
for name, model in models.items():
    y_pred = model.predict(X_test)
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test)
    else:
        # For models without predict_proba (e.g., SVC without probability=True)
        ovo_classifier = OneVsOneClassifier(model)
        ovo_classifier.fit(X_train, y_train)
        y_proba = ovo_classifier.decision_function(X_test)
    evaluate_model(y_test, y_pred, y_proba, name)

Random Forest:
Accuracy: 0.92
F1 Score (Micro): 0.92
F1 Score (Macro): 0.46208112874779544
ROC AUC (OvO): 0.4269255603687511
ROC AUC (OvR): 0.4269255603687511
              precision    recall  f1-score   support

         bot       1.00      0.27      0.43        11
      normal       0.92      1.00      0.96       181
       troll       0.00      0.00      0.00         8

    accuracy                           0.92       200
   macro avg       0.64      0.42      0.46       200
weighted avg       0.89      0.92      0.89       200


Logistic Regression:
Accuracy: 0.905
F1 Score (Micro): 0.905
F1 Score (Macro): 0.31671041119860016
ROC AUC (OvO): 0.6244456579965479
ROC AUC (OvR): 0.6244456579965479
              precision    recall  f1-score   support

         bot       0.00      0.00      0.00        11
      normal       0.91      1.00      0.95       181
       troll       0.00      0.00      0.00         8

    accuracy                           0.91       200
   macro avg       0

/Users/pratappokkharel/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/pratappokkharel/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/pratappokkharel/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

SVC:
Accuracy: 0.92
F1 Score (Micro): 0.92
F1 Score (Macro): 0.46208112874779544
ROC AUC (OvO): 0.5445465355559511
ROC AUC (OvR): 0.5445465355559511
              precision    recall  f1-score   support

         bot       1.00      0.27      0.43        11
      normal       0.92      1.00      0.96       181
       troll       0.00      0.00      0.00         8

    accuracy                           0.92       200
   macro avg       0.64      0.42      0.46       200
weighted avg       0.89      0.92      0.89       200


Random Forest:
Accuracy: 0.92
F1 Score (Micro): 0.92
F1 Score (Macro): 0.46208112874779544
ROC AUC (OvO): 0.4269255603687511
ROC AUC (OvR): 0.4269255603687511
              precision    recall  f1-score   support

         bot       1.00      0.27      0.43        11
      normal       0.92      1.00      0.96       181
       troll       0.00      0.00      0.00         8

    accuracy                           0.92       200
   macro avg       0.64      0.42     

/Users/pratappokkharel/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/pratappokkharel/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/pratappokkharel/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

SVC:
Accuracy: 0.92
F1 Score (Micro): 0.92
F1 Score (Macro): 0.46208112874779544
ROC AUC (OvO): 0.5445465355559511
ROC AUC (OvR): 0.5445465355559511
              precision    recall  f1-score   support

         bot       1.00      0.27      0.43        11
      normal       0.92      1.00      0.96       181
       troll       0.00      0.00      0.00         8

    accuracy                           0.92       200
   macro avg       0.64      0.42      0.46       200
weighted avg       0.89      0.92      0.89       200




/Users/pratappokkharel/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/pratappokkharel/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/pratappokkharel/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

# Word2vec + SMOTE

In [13]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

# Разделение данных на признаки и целевую переменную
X = np.vstack(comments_df_fin['Vector'].values)
y = comments_df_fin['Label']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Применение SMOTE к обучающей выборке
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Создание и обучение моделей
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
lr_model = LogisticRegression(max_iter=1000, random_state=42)
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
svc_model = SVC(probability=True, random_state=42)
nb_model = MultinomialNB()

models = {
    "Random Forest": rf_model,
    "Logistic Regression": lr_model,
    "Gradient Boosting": gb_model,
    "SVC": svc_model,
#    "Naive Bayes": nb_model
}

# Обучение моделей
for name, model in models.items():
    model.fit(X_train_smote, y_train_smote)

# Предсказание на тестовых данных и оценка моделей
def evaluate_model(y_test, y_pred, y_proba, model_name):
    accuracy = accuracy_score(y_test, y_pred)
    f1_micro = f1_score(y_test, y_pred, average='micro')
    f1_macro = f1_score(y_test, y_pred, average='macro')
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test_binarized = lb.transform(y_test)
    roc_auc_ovo = roc_auc_score(y_test_binarized, y_proba, multi_class='ovo')
    roc_auc_ovr = roc_auc_score(y_test_binarized, y_proba, multi_class='ovr')
    print(f"{model_name}:")
    print("Accuracy:", accuracy)
    print("F1 Score (Micro):", f1_micro)
    print("F1 Score (Macro):", f1_macro)
    print("ROC AUC (OvO):", roc_auc_ovo)
    print("ROC AUC (OvR):", roc_auc_ovr)
    print(classification_report(y_test, y_pred))
    print()

# Вывод результатов для каждой модели
for name, model in models.items():
    y_pred = model.predict(X_test)
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test)
    else:
        # For models without predict_proba (e.g., SVC without probability=True)
        ovo_classifier = OneVsOneClassifier(model)
        ovo_classifier.fit(X_train, y_train)
        y_proba = ovo_classifier.decision_function(X_test)
    evaluate_model(y_test, y_pred, y_proba, name)

/Users/pratappokkharel/anaconda3/lib/python3.11/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


Random Forest:
Accuracy: 0.89
F1 Score (Micro): 0.89
F1 Score (Macro): 0.472135955831608
ROC AUC (OvO): 0.6221793962111445
ROC AUC (OvR): 0.6221793962111445
              precision    recall  f1-score   support

         bot       0.50      0.45      0.48        11
      normal       0.93      0.96      0.94       181
       troll       0.00      0.00      0.00         8

    accuracy                           0.89       200
   macro avg       0.48      0.47      0.47       200
weighted avg       0.86      0.89      0.88       200


Logistic Regression:
Accuracy: 0.415
F1 Score (Micro): 0.415
F1 Score (Macro): 0.27315172018121747
ROC AUC (OvO): 0.5743278209173773
ROC AUC (OvR): 0.5743278209173773
              precision    recall  f1-score   support

         bot       0.08      0.55      0.14        11
      normal       0.95      0.41      0.57       181
       troll       0.06      0.38      0.10         8

    accuracy                           0.41       200
   macro avg       0.3

# Standardized Word2vec + SMOTE + 

In [15]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

# Разделение данных на признаки и целевую переменную
X = np.vstack(comments_df_fin['Vector'].values)
y = comments_df_fin['Label']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Применение SMOTE к обучающей выборке
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Стандартизация признаков
scaler = StandardScaler()
X_train_smote = scaler.fit_transform(X_train_smote)
X_test = scaler.transform(X_test)

# Создание и обучение моделей
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
lr_model = LogisticRegression(max_iter=1000, random_state=42, multi_class='multinomial', solver='lbfgs')
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
svc_model = SVC(probability=True, random_state=42)
nb_model = MultinomialNB()

models = {
    "Random Forest": rf_model,
    "Logistic Regression": lr_model,
    "Gradient Boosting": gb_model,
    "SVC": svc_model,
#    "Naive Bayes": nb_model
}

# Обучение моделей
for name, model in models.items():
    model.fit(X_train_smote, y_train_smote)

# Предсказание на тестовых данных и оценка моделей
def evaluate_model(y_test, y_pred, y_proba, model_name):
    accuracy = accuracy_score(y_test, y_pred)
    f1_micro = f1_score(y_test, y_pred, average='micro')
    f1_macro = f1_score(y_test, y_pred, average='macro')
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test_binarized = lb.transform(y_test)
    roc_auc_ovo = roc_auc_score(y_test_binarized, y_proba, multi_class='ovo')
    roc_auc_ovr = roc_auc_score(y_test_binarized, y_proba, multi_class='ovr')
    print(f"{model_name}:")
    print("Accuracy:", accuracy)
    print("F1 Score (Micro):", f1_micro)
    print("F1 Score (Macro):", f1_macro)
    print("ROC AUC (OvO):", roc_auc_ovo)
    print("ROC AUC (OvR):", roc_auc_ovr)
    print(classification_report(y_test, y_pred))
    print()

# Вывод результатов для каждой модели
for name, model in models.items():
    y_pred = model.predict(X_test)
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test)
    else:
        # For models without predict_proba (e.g., SVC without probability=True)
        ovo_classifier = OneVsOneClassifier(model)
        ovo_classifier.fit(X_train, y_train)
        y_proba = ovo_classifier.decision_function(X_test)
    evaluate_model(y_test, y_pred, y_proba, name)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/pratappokkharel/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/00/4_gxgzyx7131tsyx_ymz0hsm0000gn/T/ipykernel_6970/515049937.py", line 12, in <module>
    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/pratappokkharel/anaconda3/lib/python3.11/site-packages/imblearn/base.py", line 203, in fit_resample
  File "/Users/pratappokkharel/anaconda3/lib/python3.11/site-packages/imblearn/base.py", line 88, in fit_resample
    X : {array-like, dataframe, sparse matrix} of shape \
             ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/pratappokkharel/anaconda3/lib/python3.11/site-packages/imblearn/over_sampling/_smote/base.py", line 355, in _fit_resample
    random_state=random_state,
          ^^^^^^^^^^^^^^^^^^^^^
  File

# Word2vec + SMOTE + Features

In [69]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

# Разделение данных на признаки и целевую переменную
X = np.vstack(comments_df_fin['Vector_features'].values)
y = comments_df_fin['Label']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Применение SMOTE к обучающей выборке
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Создание и обучение моделей
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
lr_model = LogisticRegression(max_iter=1000, random_state=42, multi_class='multinomial', solver='lbfgs')
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
svc_model = SVC(probability=True, random_state=42)
nb_model = MultinomialNB()

models = {
    "Random Forest": rf_model,
    "Logistic Regression": lr_model,
    "Gradient Boosting": gb_model,
    "SVC": svc_model,
#    "Naive Bayes": nb_model
}

# Обучение моделей
for name, model in models.items():
    model.fit(X_train_smote, y_train_smote)

# Предсказание на тестовых данных и оценка моделей
def evaluate_model(y_test, y_pred, y_proba, model_name):
    accuracy = accuracy_score(y_test, y_pred)
    f1_micro = f1_score(y_test, y_pred, average='micro')
    f1_macro = f1_score(y_test, y_pred, average='macro')
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test_binarized = lb.transform(y_test)
    roc_auc_ovo = roc_auc_score(y_test_binarized, y_proba, multi_class='ovo')
    roc_auc_ovr = roc_auc_score(y_test_binarized, y_proba, multi_class='ovr')
    print(f"{model_name}:")
    print("Accuracy:", accuracy)
    print("F1 Score (Micro):", f1_micro)
    print("F1 Score (Macro):", f1_macro)
    print("ROC AUC (OvO):", roc_auc_ovo)
    print("ROC AUC (OvR):", roc_auc_ovr)
    print(classification_report(y_test, y_pred))
    print()

# Вывод результатов для каждой модели
for name, model in models.items():
    y_pred = model.predict(X_test)
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test)
    else:
        # For models without predict_proba (e.g., SVC without probability=True)
        ovo_classifier = OneVsOneClassifier(model)
        ovo_classifier.fit(X_train, y_train)
        y_proba = ovo_classifier.decision_function(X_test)
    evaluate_model(y_test, y_pred, y_proba, name)

Random Forest:
Accuracy: 0.64
F1 Score (Micro): 0.64
F1 Score (Macro): 0.36143237605691514
ROC AUC (OvO): 0.686852599321098
ROC AUC (OvR): 0.686852599321098
              precision    recall  f1-score   support

         bot       0.14      0.45      0.21        11
      normal       0.91      0.67      0.77       181
       troll       0.07      0.25      0.11         8

    accuracy                           0.64       200
   macro avg       0.37      0.46      0.36       200
weighted avg       0.83      0.64      0.71       200


Logistic Regression:
Accuracy: 0.26
F1 Score (Micro): 0.26
F1 Score (Macro): 0.22210820446114565
ROC AUC (OvO): 0.6332451479631573
ROC AUC (OvR): 0.6332451479631573
              precision    recall  f1-score   support

         bot       0.06      0.55      0.11        11
      normal       0.95      0.22      0.35       181
       troll       0.12      0.88      0.21         8

    accuracy                           0.26       200
   macro avg       0.38 

# Word2vec + SMOTE + Standardized Features


In [70]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

# Разделение данных на признаки и целевую переменную
X = np.vstack(comments_df_fin['Vector_features'].values)
y = comments_df_fin['Label']

# Выделение последних четырех признаков для стандартизации
X_additional = X[:, -4:]

# Стандартизация последних четырех признаков
scaler = StandardScaler()
X_additional_scaled = scaler.fit_transform(X_additional)

# Объединение стандартизированных признаков с остальными
X_combined = np.hstack((X[:, :-4], X_additional_scaled))

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Применение SMOTE к обучающей выборке
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Создание и обучение моделей
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
lr_model = LogisticRegression(max_iter=1000, random_state=42, multi_class='multinomial', solver='lbfgs')
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
svc_model = SVC(probability=True, random_state=42)
nb_model = MultinomialNB()

models = {
    "Random Forest": rf_model,
    "Logistic Regression": lr_model,
    "Gradient Boosting": gb_model,
    "SVC": svc_model,
#    "Naive Bayes": nb_model
}

# Обучение моделей
for name, model in models.items():
    model.fit(X_train_smote, y_train_smote)

# Предсказание на тестовых данных и оценка моделей
def evaluate_model(y_test, y_pred, y_proba, model_name):
    accuracy = accuracy_score(y_test, y_pred)
    f1_micro = f1_score(y_test, y_pred, average='micro')
    f1_macro = f1_score(y_test, y_pred, average='macro')
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test_binarized = lb.transform(y_test)
    roc_auc_ovo = roc_auc_score(y_test_binarized, y_proba, multi_class='ovo')
    roc_auc_ovr = roc_auc_score(y_test_binarized, y_proba, multi_class='ovr')
    print(f"{model_name}:")
    print("Accuracy:", accuracy)
    print("F1 Score (Micro):", f1_micro)
    print("F1 Score (Macro):", f1_macro)
    print("ROC AUC (OvO):", roc_auc_ovo)
    print("ROC AUC (OvR):", roc_auc_ovr)
    print(classification_report(y_test, y_pred))
    print()

# Вывод результатов для каждой модели
for name, model in models.items():
    y_pred = model.predict(X_test)
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test)
    else:
        # For models without predict_proba (e.g., SVC without probability=True)
        ovo_classifier = OneVsOneClassifier(model)
        ovo_classifier.fit(X_train, y_train)
        y_proba = ovo_classifier.decision_function(X_test)
    evaluate_model(y_test, y_pred, y_proba, name)

Random Forest:
Accuracy: 0.775
F1 Score (Micro): 0.775
F1 Score (Macro): 0.44004121423476267
ROC AUC (OvO): 0.6904555357813633
ROC AUC (OvR): 0.6904555357813633
              precision    recall  f1-score   support

         bot       0.19      0.45      0.27        11
      normal       0.93      0.82      0.87       181
       troll       0.14      0.25      0.18         8

    accuracy                           0.78       200
   macro avg       0.42      0.51      0.44       200
weighted avg       0.85      0.78      0.81       200


Logistic Regression:
Accuracy: 0.25
F1 Score (Micro): 0.25
F1 Score (Macro): 0.2155307994757536
ROC AUC (OvO): 0.6245253183651656
ROC AUC (OvR): 0.6245253183651656
              precision    recall  f1-score   support

         bot       0.06      0.55      0.11        11
      normal       1.00      0.20      0.34       181
       troll       0.11      0.88      0.20         8

    accuracy                           0.25       200
   macro avg       0.

# PCA (Word2vec) + SMOTE

In [73]:


# Разделение данных на признаки и целевую переменную
X = np.vstack(comments_df_fin['Vector'].values)
y = comments_df_fin['Label']

# Применение PCA для уменьшения размерности векторов Word2Vec
pca = PCA(n_components=25, random_state=42)
X_pca = pca.fit_transform(X)

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

# Применение SMOTE к обучающей выборке
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Создание и обучение моделей
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
lr_model = LogisticRegression(max_iter=1000, random_state=42, multi_class='multinomial', solver='lbfgs')
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
svc_model = SVC(probability=True, random_state=42, kernel='rbf')
nb_model = MultinomialNB()

models = {
    "Random Forest": rf_model,
    "Logistic Regression": lr_model,
    "Gradient Boosting": gb_model,
    "SVC": svc_model,
#    "Naive Bayes": nb_model
}

# Обучение моделей
for name, model in models.items():
    model.fit(X_train_smote, y_train_smote)

# Предсказание на тестовых данных и оценка моделей
def evaluate_model(y_test, y_pred, y_proba, model_name):
    accuracy = accuracy_score(y_test, y_pred)
    f1_micro = f1_score(y_test, y_pred, average='micro')
    f1_macro = f1_score(y_test, y_pred, average='macro')
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test_binarized = lb.transform(y_test)
    roc_auc_ovo = roc_auc_score(y_test_binarized, y_proba, multi_class='ovo')
    roc_auc_ovr = roc_auc_score(y_test_binarized, y_proba, multi_class='ovr')
    print(f"{model_name}:")
    print("Accuracy:", accuracy)
    print("F1 Score (Micro):", f1_micro)
    print("F1 Score (Macro):", f1_macro)
    print("ROC AUC (OvO):", roc_auc_ovo)
    print("ROC AUC (OvR):", roc_auc_ovr)
    print(classification_report(y_test, y_pred))
    print()

# Вывод результатов для каждой модели
for name, model in models.items():
    y_pred = model.predict(X_test)
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test)
    else:
        # For models without predict_proba (e.g., SVC without probability=True)
        ovo_classifier = OneVsOneClassifier(model)
        ovo_classifier.fit(X_train, y_train)
        y_proba = ovo_classifier.decision_function(X_test)
    evaluate_model(y_test, y_pred, y_proba, name)

Random Forest:
Accuracy: 0.865
F1 Score (Micro): 0.865
F1 Score (Macro): 0.5404007756948933
ROC AUC (OvO): 0.8290817200955655
ROC AUC (OvR): 0.8290817200955655
              precision    recall  f1-score   support

         bot       0.40      0.55      0.46        11
      normal       0.94      0.91      0.92       181
       troll       0.22      0.25      0.24         8

    accuracy                           0.86       200
   macro avg       0.52      0.57      0.54       200
weighted avg       0.88      0.86      0.87       200


Logistic Regression:
Accuracy: 0.29
F1 Score (Micro): 0.29
F1 Score (Macro): 0.23942284294707203
ROC AUC (OvO): 0.6525923527472473
ROC AUC (OvR): 0.6525923527472473
              precision    recall  f1-score   support

         bot       0.08      0.64      0.14        11
      normal       0.96      0.24      0.39       181
       troll       0.11      0.88      0.19         8

    accuracy                           0.29       200
   macro avg       0.

In [40]:
# PCA (Standardized Word2vec) + SMOTE

In [60]:
comments_df_fin['Vector_features'][0].size

104

In [18]:
comments_df_fin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       1000 non-null   object 
 1   author_name      999 non-null    object 
 2   text             1000 non-null   object 
 3   reply_count      1000 non-null   float64
 4   top_level        1000 non-null   object 
 5   index            1000 non-null   object 
 6   publishedAt      1000 non-null   object 
 7   updateAt         1000 non-null   object 
 8   likeCount        1000 non-null   float64
 9   Video_ID         1000 non-null   int64  
 10  Label            1000 non-null   object 
 11  Text             1000 non-null   object 
 12  Processed_Text   1000 non-null   object 
 13  Vector           1000 non-null   object 
 14  Vector_features  1000 non-null   object 
 15  Foreign_Words    1000 non-null   int64  
dtypes: float64(2), int64(2), object(12)
memory usage: 125.1+ KB


# PCA (Word2vec) + SMOTE + Features

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

# Разделение данных на признаки и целевую переменную
X = np.vstack(comments_df_fin['Vector_features'].values)
y = comments_df_fin['Label']

# Добавление дополнительных признаков 29 - 33
additional_features = comments_df_fin[['LDA_Similarity', 'Thread_Similarity', 'Videosub_Similarity',  
                                       'Sentiment_Label', 'Sentiment_Score']].values

# Выделение последних четырех признаков для стандартизации 25 - 28
X_additional = X[:, -4:]

# Применение PCA для уменьшения размерности векторов Word2Vec  0 - 24
pca = PCA(n_components=25, random_state=42)
X_pca = pca.fit_transform(X[:, :-4])

# Стандартизация последних четырех признаков
#scaler = StandardScaler()
#X_additional_scaled = scaler.fit_transform(X_additional)

# Объединение стандартизированных признаков с остальными
#X_combined = np.hstack((X_pca, X_additional))
X_combined = np.hstack((X_pca, X_additional, additional_features))

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Применение SMOTE к обучающей выборке
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Создание и обучение моделей
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
lr_model = LogisticRegression(max_iter=1000, random_state=42, multi_class='multinomial', solver='lbfgs')
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
svc_model = SVC(probability=True, random_state=42)
nb_model = MultinomialNB()

models = {
    "Random Forest": rf_model,
    "Logistic Regression": lr_model,
    "Gradient Boosting": gb_model,
    "SVC": svc_model,
#    "Naive Bayes": nb_model
}

# Обучение моделей
for name, model in models.items():
    model.fit(X_train_smote, y_train_smote)

# Предсказание на тестовых данных и оценка моделей
def evaluate_model(y_test, y_pred, y_proba, model_name):
    accuracy = accuracy_score(y_test, y_pred)
    f1_micro = f1_score(y_test, y_pred, average='micro')
    f1_macro = f1_score(y_test, y_pred, average='macro')
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test_binarized = lb.transform(y_test)
    roc_auc_ovo = roc_auc_score(y_test_binarized, y_proba, multi_class='ovo')
    roc_auc_ovr = roc_auc_score(y_test_binarized, y_proba, multi_class='ovr')
    print(f"{model_name}:")
    print("Accuracy:", accuracy)
    print("F1 Score (Micro):", f1_micro)
    print("F1 Score (Macro):", f1_macro)
    print("ROC AUC (OvO):", roc_auc_ovo)
    print("ROC AUC (OvR):", roc_auc_ovr)
    print(classification_report(y_test, y_pred))
    print()

# Вывод результатов для каждой модели
for name, model in models.items():
    y_pred = model.predict(X_test)
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test)
    else:
        # For models without predict_proba (e.g., SVC without probability=True)
        ovo_classifier = OneVsOneClassifier(model)
        ovo_classifier.fit(X_train, y_train)
        y_proba = ovo_classifier.decision_function(X_test)
    evaluate_model(y_test, y_pred, y_proba, name)
    
# Оценка значимости признаков
def feature_importance(model, model_name, X_train, y_train):
    print(f"{model_name} Feature Importance:")
    if model_name == "Random Forest" or model_name == "Gradient Boosting":
        importances = model.feature_importances_
        sorted_indices = np.argsort(importances)[::-1]
        for idx in sorted_indices:
            print(f"Feature {idx}: {importances[idx]}")
    elif model_name == "Logistic Regression":
        importances = np.abs(model.coef_[0])
        sorted_indices = np.argsort(importances)[::-1]
        for idx in sorted_indices:
            print(f"Feature {idx}: {importances[idx]}")
    elif model_name == "SVC":
        if hasattr(model, "coef_"):
            importances = np.abs(model.coef_[0])
            sorted_indices = np.argsort(importances)[::-1]
            for idx in sorted_indices:
                print(f"Feature {idx}: {importances[idx]}")
        else:
            result = permutation_importance(model, X_train, y_train, n_repeats=10, random_state=42, n_jobs=2)
            sorted_indices = result.importances_mean.argsort()[::-1]
            for idx in sorted_indices:
                print(f"Feature {idx}: {result.importances_mean[idx]}")
    print()

# Вывод значимости признаков для каждой модели
for name, model in models.items():
    feature_importance(model, name, X_train_smote, y_train_smote)


/Users/pratappokkharel/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Random Forest:
Accuracy: 0.91
F1 Score (Micro): 0.91
F1 Score (Macro): 0.517117117117117
ROC AUC (OvO): 0.7234060109694491
ROC AUC (OvR): 0.7234060109694491
              precision    recall  f1-score   support

         bot       0.67      0.55      0.60        11
      normal       0.93      0.97      0.95       181
       troll       0.00      0.00      0.00         8

    accuracy                           0.91       200
   macro avg       0.53      0.51      0.52       200
weighted avg       0.88      0.91      0.89       200


Logistic Regression:
Accuracy: 0.235
F1 Score (Micro): 0.235
F1 Score (Macro): 0.19554274671071115
ROC AUC (OvO): 0.6666071567090138
ROC AUC (OvR): 0.6666071567090138
              precision    recall  f1-score   support

         bot       0.09      0.64      0.16        11
      normal       0.90      0.19      0.32       181
       troll       0.06      0.62      0.11         8

    accuracy                           0.23       200
   macro avg       0.3

Выходит, из дополнительных признаков знаичмыми являются только - 'LDA_Similarity', 'Thread_Similarity', 'Videosub_Similarity', 'Sentiment_Label', 'Sentiment_Score'

In [16]:
comments_df_fin.columns

Index(['Unnamed: 0', 'author_name', 'text', 'reply_count', 'top_level',
       'index', 'publishedAt', 'updateAt', 'likeCount', 'Video_ID', 'Label',
       'Text', 'Processed_Text', 'Vector', 'Vector_features', 'Foreign_Words'],
      dtype='object')

In [ ]:
comments_df_fin[]